In [1]:
from mistralai import Mistral
import os
from dotenv import load_dotenv
from mistralai.models import UserMessage, SystemMessage

load_dotenv()

True

In [27]:
system_prompt = SystemMessage(
    content="""You are an expert Connect Four player.

GAME RULES:
- 7 columns (numbered 0-6), 6 rows
- Pieces drop to the lowest empty row in chosen column
- Win by connecting 4 pieces horizontally, vertically, or diagonally
- You are piece 1, opponent is piece 2

STRATEGY (check in this order):
1. If YOU can win this turn, play that winning move
2. If OPPONENT can win next turn, block them
3. Otherwise, play strategically (center columns are strongest)

OUTPUT FORMAT:
Return ONLY valid JSON: {"col": integer}
No explanations. No other text.

EXAMPLES:
Example 1 - Take the win:
Board: Row5: 1 1 1 0 0 0 0
Analysis: You can win by playing column 3
Answer: {{"col": 3}}

Example 2 - Block opponent:
Board: Row5: 2 2 2 0 0 0 0
Analysis: Opponent wins if you don't block column 3
Answer: {{"col": 3}}

""")

In [39]:
def create_prompt(board, piece) -> UserMessage:
    s = "CURRENT BOARD (0=empty, 1=YOU, 2=OPPONENT):\n"
    s += "Col : 0 1 2 3 4 5 6\n"
    # print col i
    for i, r in enumerate(board):
        s += f"Row{i}: " + " ".join(map(str, r)) + "\n"

    s += f"""
You are piece {piece} (shown as 1 above).
Opponent is piece {3 - piece} (shown as 2 above).

ANALYZE:
1. Can YOU win this turn? If yes, play that column.
2. Can OPPONENT win next turn? If yes, block that column.
3. Otherwise choose strategically.


Return ONLY: {{"col": integer}}
"""
    return UserMessage(content=s)

In [40]:
board = [
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 2],
            [0, 0, 0, 0, 0, 0, 2],
            [1, 1, 0, 0, 0, 0, 2],
]
piece = 1
prompt = create_prompt(board, piece)

In [41]:
print(prompt.content)

CURRENT BOARD (0=empty, 1=YOU, 2=OPPONENT):
Col : 0 1 2 3 4 5 6
Row0: 0 0 0 0 0 0 0
Row1: 0 0 0 0 0 0 0
Row2: 0 0 0 0 0 0 0
Row3: 0 0 0 0 0 0 2
Row4: 0 0 0 0 0 0 2
Row5: 1 1 0 0 0 0 2

You are piece 1 (shown as 1 above).
Opponent is piece 2 (shown as 2 above).

ANALYZE:
1. Can YOU win this turn? If yes, play that column.
2. Can OPPONENT win next turn? If yes, block that column.
3. Otherwise choose strategically.


Return ONLY: {"col": integer}



In [31]:
with Mistral(
    api_key=os.environ["MISTRAL_API_KEY"],
) as mistral:
    models = mistral.models.list().data
    print("Available models:", "\n".join(m.name for m in models))

Available models: mistral-medium-2505
mistral-medium-2508
mistral-medium-2508
mistral-medium-2508
open-mistral-nemo
open-mistral-nemo
open-mistral-nemo
open-mistral-nemo
mistral-large-2411
pixtral-large-2411
pixtral-large-2411
pixtral-large-2411
codestral-2508
codestral-2508
devstral-small-2507
devstral-medium-2507
devstral-2512
devstral-2512
devstral-2512
devstral-2512
labs-devstral-small-2512
labs-devstral-small-2512
mistral-small-2506
mistral-small-2506
labs-mistral-small-creative
magistral-medium-2509
magistral-medium-2509
magistral-small-2509
magistral-small-2509
voxtral-mini-2507
voxtral-mini-2507
voxtral-small-2507
voxtral-small-2507
mistral-large-2512
mistral-large-2512
ministral-3b-2512
ministral-3b-2512
ministral-8b-2512
ministral-8b-2512
ministral-14b-2512
ministral-14b-2512
open-mistral-7b
open-mistral-7b
open-mistral-7b
pixtral-12b-2409
pixtral-12b-2409
pixtral-12b-2409
ministral-3b-2410
ministral-8b-2410
codestral-2501
codestral-2501
codestral-2501
mistral-small-2501
mist

In [42]:
with Mistral(
    api_key=os.environ["MISTRAL_API_KEY"],
) as mistral:
    response = mistral.chat.complete(
        model="devstral-2512",
        response_format={"type": "json_object"},
        messages=[system_prompt, prompt],
    )
    print("Mistral's Move Response:", response.choices[0].message.content)

Mistral's Move Response: {"col": 3}
